# 1. 爬虫设计

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import requests
import re
import talib as ta

In [11]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示列不换行
pd.set_option('expand_frame_repr',False)
#显示行数
pd.set_option('display.max_rows', None)

url='http://66.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124009640769137954641_1607884920838&pn=1&pz=100000&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80,m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1607884920931'
# 将pz设置一个较大的值即可获取网页所有数据避免翻页
html=requests.get(url)
datas=re.compile('({"f1":.*?})',re.S).findall(html.text)
labels = ['序号','代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量(手)', '成交额', '振幅', '最高', '最低', '今天', '昨收', '量比', '换手率', '市盈率(动态)',
              '市净率']
row_index = 0
df=pd.DataFrame(columns=labels)
for data in datas:

    dict = eval(data)
    row=[row_index,str(dict['f12']),str(dict['f14']),dict['f2'],dict['f3'],dict['f4'],dict['f5'],dict['f6'],dict['f7'],dict['f15'],dict['f16'],dict['f17'],dict['f18'],dict['f10'],dict['f8'],dict['f9'],dict['f23']]
    df_row=[row]
    df=df.append({'序号':row_index,'代码':dict['f12'], '名称':dict['f14'], '最新价':dict['f2'], '涨跌幅':dict['f3'], '涨跌额':dict['f4'], '成交量(手)':dict['f5'], '成交额':dict['f6'], '振幅':dict['f7'], '最高':dict['f15'], '最低':dict['f16'], '今天':dict['f17'], '昨收':dict['f18'], '量比':dict['f10'], '换手率':dict['f8'], '市盈率(动态)':dict['f9'],'市净率':dict['f23']},ignore_index=True)
    row_index += 1
print(df)


        序号      代码        名称      最新价     涨跌幅     涨跌额   成交量(手)          成交额      振幅       最高       最低      今天       昨收     量比    换手率  市盈率(动态)      市净率
0        0  688510       N航亚    37.18  355.08   29.01   489326  1.72351e+09  182.37     45.5     30.6      35     8.17      -  98.96   182.68    10.45
1        1  000995      皇台酒业       31  314.99   23.53   209699  6.11192e+08   80.32       31       25      25     7.47   2.28  11.82   1030.5    53.11
2        2  688608       N恒玄      362  123.36  199.93   158149  5.94139e+09   30.59      403   353.42     391   162.07      -  68.82   278.46     8.03
3        3  300106      西部牧业    15.17   13.46     1.8   475727  6.87301e+08    17.2    15.65    13.35   13.47    13.37   1.56  22.51   377.32      5.6
4        4  300208      青岛中程    14.35   12.46    1.59   849974  1.22043e+09   13.87    15.28    13.51   13.99    12.76   1.58  12.37  -530.68     6.14
5        5  300089      文化长城     5.69   11.79     0.6   195061  1.11987e+08   18.66     6.11  

# 数据处理

In [12]:
df.head()

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量(手),成交额,振幅,最高,最低,今天,昨收,量比,换手率,市盈率(动态),市净率
0,0,688510,N航亚,37.18,355.08,29.01,489326,1.72351e+09,182.37,45.5,30.6,35,8.17,-,98.96,182.68,10.45
1,1,000995,皇台酒业,31,314.99,23.53,209699,6.11192e+08,80.32,31,25,25,7.47,2.28,11.82,1030.5,53.11
2,2,688608,N恒玄,362,123.36,199.93,158149,5.94139e+09,30.59,403,353.42,391,162.07,-,68.82,278.46,8.03
3,3,300106,西部牧业,15.17,13.46,1.8,475727,6.87301e+08,17.2,15.65,13.35,13.47,13.37,1.56,22.51,377.32,5.6
4,4,300208,青岛中程,14.35,12.46,1.59,849974,1.22043e+09,13.87,15.28,13.51,13.99,12.76,1.58,12.37,-530.68,6.14


In [13]:
df.columns

Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量(手)', '成交额', '振幅', '最高',
       '最低', '今天', '昨收', '量比', '换手率', '市盈率(动态)', '市净率'],
      dtype='object')

In [14]:
for i in df.columns:
    df.loc[df[i] == '-', i] = np.nan

In [15]:
df.head()

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量(手),成交额,振幅,最高,最低,今天,昨收,量比,换手率,市盈率(动态),市净率
0,0,688510,N航亚,37.18,355.08,29.01,489326,1.72351e+09,182.37,45.5,30.6,35,8.17,NaN,98.96,182.68,10.45
1,1,000995,皇台酒业,31,314.99,23.53,209699,6.11192e+08,80.32,31,25,25,7.47,2.28,11.82,1030.5,53.11
2,2,688608,N恒玄,362,123.36,199.93,158149,5.94139e+09,30.59,403,353.42,391,162.07,NaN,68.82,278.46,8.03
3,3,300106,西部牧业,15.17,13.46,1.8,475727,6.87301e+08,17.2,15.65,13.35,13.47,13.37,1.56,22.51,377.32,5.6
4,4,300208,青岛中程,14.35,12.46,1.59,849974,1.22043e+09,13.87,15.28,13.51,13.99,12.76,1.58,12.37,-530.68,6.14


In [16]:
import datetime
today=datetime.date.today()

In [17]:
df['Date'] = today   #加上每天的日期

In [18]:
df.head()

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量(手),成交额,振幅,最高,最低,今天,昨收,量比,换手率,市盈率(动态),市净率,Date
0,0,688510,N航亚,37.18,355.08,29.01,489326,1.72351e+09,182.37,45.5,30.6,35,8.17,NaN,98.96,182.68,10.45,2020-12-16
1,1,000995,皇台酒业,31,314.99,23.53,209699,6.11192e+08,80.32,31,25,25,7.47,2.28,11.82,1030.5,53.11,2020-12-16
2,2,688608,N恒玄,362,123.36,199.93,158149,5.94139e+09,30.59,403,353.42,391,162.07,NaN,68.82,278.46,8.03,2020-12-16
3,3,300106,西部牧业,15.17,13.46,1.8,475727,6.87301e+08,17.2,15.65,13.35,13.47,13.37,1.56,22.51,377.32,5.6,2020-12-16
4,4,300208,青岛中程,14.35,12.46,1.59,849974,1.22043e+09,13.87,15.28,13.51,13.99,12.76,1.58,12.37,-530.68,6.14,2020-12-16


# 数据存储

In [19]:
df.to_csv(f'A股_{today}.csv',encoding="utf-8",index=False) #储存每日最新的数据

In [20]:
df.to_csv('A股_All.csv', encoding="utf-8",index=False, mode = "w") # 储存所有数据，每次运行数据将自动更新在该csv文件中

In [21]:
df.to_excel(f'C:/A股数据/A股_{today}.xlsx') # 存为Excel格式的数据

# 统计分析

In [28]:
df.loc[df['成交量(手)'] == df['成交量(手)'].max(), ['代码', '名称']]

,代码,名称
203,002610,爱康科技


In [33]:
df.loc[df['成交量(手)'] == df['成交量(手)'].min(), ['代码', '名称']]

,代码,名称
26,003020,立方制药


In [29]:
df.loc[df['涨跌幅'] == df['涨跌幅'].max(), ['代码', '名称']]

,代码,名称
0,688510,N航亚


In [34]:
df.loc[df['涨跌幅'] == df['涨跌幅'].min(), ['代码', '名称']]

,代码,名称
4267,300290,荣科科技


In [30]:
df.loc[df['换手率'] == df['换手率'].max(), ['代码', '名称']]

,代码,名称
0,688510,N航亚


In [35]:
df.loc[df['换手率'] == df['换手率'].min(), ['代码', '名称']]

,代码,名称
97,600877,ST电能
2248,000617,中油资本


In [31]:
df.loc[df['市盈率(动态)'] == df['市盈率(动态)'].max(), ['代码', '名称']]

,代码,名称
2348,300006,莱美药业


In [36]:
df.loc[df['市盈率(动态)'] == df['市盈率(动态)'].min(), ['代码', '名称']]

,代码,名称
4141,688126,沪硅产业-U


In [32]:
df.loc[df['市净率'] == df['市净率'].max(), ['代码', '名称']]

,代码,名称
1236,000670,*ST盈方


In [37]:
df.loc[df['市净率'] == df['市净率'].min(), ['代码', '名称']]

,代码,名称
1198,002188,ST巴士
